In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df_all = pd.read_csv('all-jobs.csv')
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3205 entries, 0 to 3204
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   AY           3205 non-null   object
 1   Institution  3205 non-null   object
 2   Position     3205 non-null   object
 3   Address      3205 non-null   object
 4   Link         3061 non-null   object
 5   TrackType    3205 non-null   object
 6   DateAdded    3205 non-null   object
dtypes: object(7)
memory usage: 175.4+ KB


In [3]:
df_combined = pd.read_csv('combined-jobs.csv')
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3255 entries, 0 to 3254
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AY                3255 non-null   object 
 1   Institution       3255 non-null   object 
 2   Position          3255 non-null   object 
 3   Address           3255 non-null   object 
 4   Link              3098 non-null   object 
 5   TrackType         3255 non-null   object 
 6   DateAdded         3255 non-null   object 
 7   DateTimeObj       3255 non-null   object 
 8   Latitude          3255 non-null   float64
 9   Longitude         3255 non-null   float64
 10  FormattedAddress  3255 non-null   object 
 11  City              3186 non-null   object 
 12  County            2927 non-null   object 
 13  State             3232 non-null   object 
 14  StateCode         3120 non-null   object 
 15  Country           3254 non-null   object 
 16  CountryCode       3255 non-null   object 


## Show Difference Between Two DataFrames

In [9]:
df_diff = df_all.merge(df_combined, indicator = True, how='left').loc[lambda x : x['_merge']!='both']
df_diff

,AY,Institution,Position,Address,Link,TrackType,DateAdded,DateTimeObj,Latitude,Longitude,FormattedAddress,City,County,State,StateCode,Country,CountryCode,_merge


In [10]:
df_diff.Institution.values.tolist()

[]

## Create TrackType Data SubSet

Write a new file with per date sum of job type post counts:

{date, posts: INT total of TT, type: "TT"},
{date, posts: INT total of NTT, type: "NTT"},
{date, posts: INT total of Unavailable, type: "Unavailable"},

In [4]:
def create_subset(df, subset_col_1, subset_col_dates):
  '''
  # @create_subset: Create subsets of TrackType data

  # Params
  - df: pandas DataFrame. Data set with all parameters of interest
  - subset_col: String. Column of interest.
  
  ## Return
  - new_df: pandas DataFrame. New dataframe with following structure per row: 
    - date: DateTime Obj of posting date.
    - type: String. Type of track: TT, NTT, Unavailable.
    - posts: Integer. Total sum of posts of that type of job on that specific date.
  '''
  uniq_subset_col_values = df[subset_col_1].unique()
  uniq_subset_col_dates = sorted(df[subset_col_dates].apply(lambda d: datetime.strptime(d,'%m/%d/%Y')).unique())
  list_unique_dates_sorted = []
  for date in uniq_subset_col_dates:
    list_unique_dates_sorted.append(date.strftime("%m/%d/%Y"))

  list_record_rows_per_date = []
  list_record_rows_flat = []
  for date_str in list_unique_dates_sorted:
    if date_str != '01/01/1900':
      date_slice = df.loc[df.DateAdded == date_str]
      list_record_rows_per_date.append({
        'date': date_str,
        'datetimeObj': datetime.strptime(date_str, '%m/%d/%Y'),
        'TT': len(date_slice.loc[date_slice.TrackType == 'TT']),
        'NTT': len(date_slice.loc[date_slice.TrackType == 'NTT']),
        'Unavailable': len(date_slice.loc[date_slice.TrackType == 'Unavailable']),
        'Total': len(date_slice.loc[date_slice.TrackType == 'TT'])+len(date_slice.loc[date_slice.TrackType == 'NTT'])+len(date_slice.loc[date_slice.TrackType == 'Unavailable'])
      })

      for tt in uniq_subset_col_values:
        tt_date_slice = date_slice.loc[date_slice.TrackType == tt]
        list_record_rows_flat.append({
          'date': date_str,
          'datetimeObj': datetime.strptime(date_str, '%m/%d/%Y'),
          'type': tt,
          'posts': len(tt_date_slice)
        })
  df_new_flat = pd.DataFrame(list_record_rows_flat)
  df_new_per_date = pd.DataFrame(list_record_rows_per_date)
  # Add moving averages
  df_new_flat['Overall_EMA'] = df_new_flat['posts'].ewm(span=7).mean()        # Exponential Moving Average
  df_new_per_date['Overall_EMA'] = df_new_per_date['Total'].ewm(span=7).mean()        # Exponential Moving Average
  return {
    'df_new_flat': df_new_flat,
    'df_new_per_date': df_new_per_date,
  }


In [5]:
df__tt_per_date_flat = create_subset(df_combined, subset_col_1='TrackType', subset_col_dates='DateAdded')
df__tt_per_date_flat['df_new_per_date'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076 entries, 0 to 1075
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1076 non-null   object        
 1   datetimeObj  1076 non-null   datetime64[ns]
 2   TT           1076 non-null   int64         
 3   NTT          1076 non-null   int64         
 4   Unavailable  1076 non-null   int64         
 5   Total        1076 non-null   int64         
 6   Overall_EMA  1076 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 59.0+ KB


In [6]:
df__tt_per_date_flat['df_new_per_date'].head()

,date,datetimeObj,TT,NTT,Unavailable,Total,Overall_EMA
0,09/15/2012,2012-09-15,0,1,0,1,1.000000
1,01/15/2013,2013-01-15,0,1,0,1,1.000000
2,01/20/2013,2013-01-20,0,1,0,1,1.000000
3,01/23/2013,2013-01-23,2,0,0,2,1.365714
4,01/24/2013,2013-01-24,1,0,0,1,1.245839


In [7]:
df__tt_per_date_flat['df_new_flat'].to_csv('tt-per-date-flat.csv', index=False)

In [8]:
df__tt_per_date_flat['df_new_per_date'].to_csv('tt-per-date.csv', index=False)